In [1]:
import re
import json
from pathlib import Path

import pandas as pd

# Generating CSV with data

In [2]:
raw_path = Path("raw_data")
data = list()
for fl in raw_path.glob("*.json"):
    with open(fl) as f:
        raw_data = json.load(f)
    for d in raw_data:
        del d["mfv"]["chat_history"]
        del d["mfq"]["chat_history"]
    data.extend(raw_data)


In [12]:
data

[{'agent': 'Gemini Pro',
  'id': 0,
  'condition': 'qv',
  'mfq': {'part1': '1. 3\n2. 4\n3. 5\n4. 5\n5. 4\n6. 0\n7. 3\n8. 4\n9. 2\n10. 5\n11. 5\n12. 4\n13. 3\n14. 3\n15. 5\n16. 4',
   'part2': '5.44.21.41.55.43.45.54.22.21.42.24.32.45.43.2',
   'part1_order': [['decency', 'purity'],
    ['betray', 'ingroup'],
    ['treated', 'fairness'],
    ['loyalty', 'ingroup'],
    ['unfairly', 'fairness'],
    ['math', None],
    ['chaos', 'authority'],
    ['respect', 'authority'],
    ['god', 'purity'],
    ['weak', 'harm'],
    ['cruel', 'harm'],
    ['emotionally', 'harm'],
    ['lovecountry', 'ingroup'],
    ['disgusting', 'purity'],
    ['rights', 'fairness'],
    ['traditions', 'authority']],
   'part2_order': [['team', 'ingroup'],
    ['kidrespect', 'authority'],
    ['rich', 'fairness'],
    ['unnatural', 'purity'],
    ['compassion', 'harm'],
    ['soldier', 'authority'],
    ['good', None],
    ['fairly', 'fairness'],
    ['history', 'ingroup'],
    ['harmlessdg', 'purity'],
    ['anima

In [3]:
clean_data = list()

for answer in data:
    mfq = answer["mfq"]
    part1_order = [x[0] for x in mfq["part1_order"]]
    part2_order = [x[0] for x in mfq["part2_order"]]
    mfv = answer["mfv"]

    clean_data.append(
        [
            answer["agent"],
            answer["id"],
            answer["condition"],
            mfq["part1"],
            part1_order,
            mfq["part2"],
            part2_order,
            mfv["mfv"],
            mfv["mfv_codes"],
        ]
    )

df = pd.DataFrame(
    clean_data,
    columns=[
        "agent",
        "id",
        "condition",
        "mfq_part1",
        "part1_order",
        "mfq_part2",
        "part2_order",
        "mfv",
        "mfv_codes",
    ],
)

df.to_csv("data/raw_completions.csv", index=False)

In [7]:
class BaseSanitizer():
    def sanitize(self):
        raise NotImplementedError("Subclass must implement abstract method")

    def validate_mfq_answer(self, answer):
        # convert matches to int
        matches = [int(x) for x in self.sanitize(answer)]
        if len(matches) not in [16,]:
            return False
        elif len(matches) == 32:
            print("WARNING: 32 matches found!")
            return False
        # check if numbers fall  between 0 and 5
        if all([x >= 0 and x <= 5 for x in matches]):
            return matches
        else:
            return False
        
    def validate_mfv_answer(self, answer):
        # convert matches to int
        matches = [int(x) for x in self.sanitize(answer)]
        if len(matches) != 68:
            return False
        # check if numbers fall between 0 and 5
        if all([x >= 1 and x <= 5 for x in matches]):
            return matches
        else:
            return False


class Sanitizer(BaseSanitizer):
    pat = r"(\d+\.\s*?)?(?P<num>\d(?!\.\d))\s*?(,|\n|$|\]|`)"
    
    def sanitize(self, answer):
        answers = re.finditer(self.pat, answer)
        # extract num groups only
        return [x.group("num") for x in answers]


In [8]:
def get_sanitizer(agent):
    return Sanitizer()

In [18]:
# create sequential id for each agent
df["id"] = df.groupby("agent").cumcount()

In [20]:
mfq_dfs = list()
mfv_dfs = list()

errors = list()

for i, row in df.iterrows():
    agent = row["agent"]
    sanitizer = get_sanitizer(agent)
    p1 = sanitizer.validate_mfq_answer(row["mfq_part1"])
    p2 = sanitizer.validate_mfq_answer(row["mfq_part2"])
    mfv = sanitizer.validate_mfv_answer(row["mfv"])

    if p1 is False or p2 is False or mfv is False:
        errors.append(
            {
                "agent": agent,
                "id": row["id"],
                "condition": row["condition"],
                "mfq_part1": row["mfq_part1"],
                "mfq_part2": row["mfq_part2"],
                "mfv": row["mfv"],
            }
        )
        continue

    mfv_data = {
        "agent": agent,
        "id": row["id"],
        "condition": row["condition"],
        "mfv": mfv,
        "mfv_codes": row["mfv_codes"],
    }

    mfq_data = {
        "agent": [agent] * 32,
        "id": [row["id"]] * 32,
        "condition": [row["condition"]] * 32,
        "answer": p1 + p2,
        "code": row["part1_order"] + row["part2_order"],
    }

    mfq_dfs.append(pd.DataFrame(mfq_data))
    mfv_dfs.append(pd.DataFrame(mfv_data))

mfq = pd.concat(mfq_dfs)
mfq.to_csv("data/mfq.csv", index=False)
mfv = pd.concat(mfv_dfs)
mfv.to_csv("data/mfv.csv", index=False)

In [21]:
mfq

,agent,id,condition,answer,code
0,Gemini Pro,13,vq,4,chaos
1,Gemini Pro,13,vq,3,god
2,Gemini Pro,13,vq,3,treated
3,Gemini Pro,13,vq,3,loyalty
4,Gemini Pro,13,vq,5,decency
...,...,...,...,...,...
27,Claude 2.1,197,vq,3,harmlessdg
28,Claude 2.1,197,vq,3,kidrespect
29,Claude 2.1,197,vq,2,unnatural
30,Claude 2.1,197,vq,2,kill


In [22]:
mfq[["agent", "id", "condition"]].drop_duplicates().groupby(["agent", "condition"]).count()

id
agent      condition     
Claude 2.1 qv          99
           vq          59
GPT-4      qv         100
           vq          89
Gemini Pro qv           1
           vq           2

Checking Gemini pattern

In [23]:
class AnswerCounter():
    pat = r"(\d+\.\s*?)?(?P<num>\d(?!\.\d))\s*?(,|\n|$|\]|`)"
    def __init__(self, answer):
        self.matches = self.sanitize(answer)
        self.count = len(self.matches)

    def sanitize(self, answer):
        answers = re.finditer(self.pat, answer)
        # extract num groups only
        return [int(x.group("num")) for x in answers]

    def validate_mfq_answer_scale(self):
        # check if numbers fall  between 0 and 5
        if all([x >= 0 and x <= 5 for x in self.matches]):
            return True
        else :
            return False
        
    def validate_mfv_answer(self):
        if all([x >= 1 and x <= 5 for x in self.matches]):
            return True
        else:
            return False

In [24]:
mfq_1 = []
mfq1_scale = []
mfq_2 = []
mfq2_scale = []
mfv = []
mfv_scale = []

for i, row in df.query("agent == 'Gemini Pro'").iterrows():
    # count number of answers
    counter = AnswerCounter(row["mfq_part1"])
    mfq_1.append(counter.count)
    mfq1_scale.append(counter.validate_mfq_answer_scale())
    counter = AnswerCounter(row["mfq_part2"])
    mfq_2.append(counter.count)
    mfq2_scale.append(counter.validate_mfq_answer_scale())
    counter = AnswerCounter(row["mfv"])
    mfv.append(counter.count)
    mfv_scale.append(counter.validate_mfv_answer())

# print number of valid scales for each item
print(sum(mfq1_scale))
print(sum(mfq2_scale))
print(sum(mfv_scale))
# print number of mfq answers != 16 and mfv answers != 68
print(sum([x != 16 for x in mfq_1]))
print(sum([x != 16 for x in mfq_2]))
print(sum([x != 68 for x in mfv]))


193
198
193
43
161
158
